In [1]:
# thread settings
import os
os.environ["OMP_NUM_THREADS"] = '1'
os.environ["OPENBLAS_NUM_THREADS"] = '1'
os.environ["MKL_NUM_THREADS"] = '1'
os.environ["VECLIB_MAXIMUM_THREADS"] = '1'
os.environ["NUMEXPR_NUM_THREADS"] = '1'

import multiprocessing as mp
from run_experiments import run_experiment
import time
import numpy as np
import yaml
import argparse
import dataset

import yaml
import dataset
import numpy as np
import math
import multipref
import metrics
import scipy.io as sio
import logging
import pickle
import cvxpy as cp
import utils

In [2]:
with open('./configs/config_color_large.yaml') as f:
        config = yaml.safe_load(f)

save_root = config['save_root']
exp_name = config['exp_name']
n_cores = config['n_cores']
pm = config['pm']
methods = config['methods']

In [3]:
now_ms = round(time.time() * 100)
np.random.seed(now_ms % 2**32)

params = [(pm, methods, '{}_{}_{}'.format(save_root + exp_name, now_ms, i),
            np.random.randint(2**31)) for i in range(n_cores)]

In [19]:
params[0]

({'fun': 'color_experiment',
  'T': 1,
  'N': 48,
  'n_splits': 5,
  'loss_fun': 'hinge',
  'hyper_sweep': [],
  'm_train': 300,
  'm_train_res': 5},
 ['frobM_l2v', 'I_l2v', 'frobM_l2v_single'],
 './save/color_experiment_N48_170413727977_0',
 2065263682)

In [5]:
m_train = pm['m_train']
m_train_res = pm['m_train_res']
N = pm['N']
if m_train_res is None:
    m_train_vec = np.array(m_train)
else:
    m_train_vec = np.arange(start=m_train_res, stop=m_train + 1, step=m_train_res)

In [6]:
if 'frobM_l2v_single' in methods:
        n_grouped_methods = len(methods)-1  # FIXME: why?
else:
    n_grouped_methods = len(methods)

In [7]:
results = {'methods': methods,
               'N': N,
               'm_train_vec': m_train_vec,
               'relative_M_error': np.zeros((len(m_train_vec), pm['T'], n_grouped_methods)),
               'scaled_M_error': np.zeros((len(m_train_vec), pm['T'], n_grouped_methods)),
               'test_accuracy': np.zeros((len(m_train_vec), pm['T'], n_grouped_methods)),
               'test_accuracy_individual': np.zeros((len(m_train_vec), pm['T'], N, n_grouped_methods)),
               'M': np.zeros((len(m_train_vec), pm['T'], n_grouped_methods, 3, 3)),
               'V': np.zeros((len(m_train_vec), pm['T'], n_grouped_methods, 3, N))
               }

results_single = {'relative_M_error': np.zeros((len(m_train_vec), pm['T'], N)),
                    'scaled_M_error': np.zeros((len(m_train_vec), pm['T'], N)),
                    'test_accuracy': np.zeros((len(m_train_vec), pm['T'], N)),
                    'M': np.zeros((len(m_train_vec), pm['T'], N, 3, 3)),
                    'V': np.zeros((len(m_train_vec), pm['T'], N, 3))
                    }

In [8]:
results['M'].shape

(60, 1, 2, 3, 3)

In [9]:
results_single['M'].shape

(60, 1, 48, 3, 3)

In [10]:
color_dataset = dataset.Dataset(dataset_type='Color',
            color_path='./CPdata.mat', N=N)

In [11]:
data = color_dataset.getAllData()
X = data['X']
max_x_norm = max(np.linalg.norm(X, axis=0))

In [12]:
S_train, S_test, Y_train, Y_test = color_dataset.getTrainTestSplit(
            train_size = m_train * N)

In [13]:
len(S_train)

14400

In [20]:
m_train_vec.shape

(60,)